#### 1. Contexto de Negócio (Business Understanding)
Objetivo: Criar uma visão unificada e calculada do pedido para determinar o Lucro Bruto Unitário (Margem de Contribuição) por transação. Esta métrica é vital para um Marketplace Two-Sided (conectando lojas, clientes e motoristas) porque permite à gestão otimizar preços, comissões, e repasses (P&L), garantindo a sustentabilidade da plataforma.

Desafio: O dado do Marketplace é naturalmente disperso em três fontes principais (Pedidos, Pagamentos e Logística), que precisam ser unidas e padronizadas.

#### 2. Camada BRONZE: Ingestão e Padronização (Data Preparation - Raw)
A Camada Bronze serve como a Zona de Aterrissagem (Landing Zone) e é a primeira etapa de preparação, garantindo a fidelidade dos dados brutos e a auditoria.

 Justificativas Técnicas
Imutabilidade: Os dados são carregados exatamente como vieram dos CSVs, preservando o histórico da fonte.

Correção de Chaves: O foco principal aqui foi resolver o [UNRESOLVED_COLUMN] e o desalinhamento das chaves do marketplace:

Mapeamento de delivery_order_id e payment_order_id para o campo canônico order_id.

Correção de case-sensitivity (ex: DELIVERED vs delivered) para evitar perdas de dados nas camadas superiores.

#### Etapas & Benefícios

| Ação | Etapa Técnica | Benefício Estratégico |
| :--- | :--- | :--- |
| Ingestão | Uso de COPY INTO e FILEFORMAT = CSV. | Auditabilidade: Cria tabelas rastreáveis (Bronze), permitindo reverter ou auditar qualquer transformação futura contra a fonte original. |
| Padronização | Uso de SELECT * AS novo_nome nos comandos COPY INTO. | Coerência: Garante que todas as tabelas de fatos usem uma chave de junção comum (order_id) antes da modelagem complexa. |
| Data Type | Tipagem inicial como STRING. | Resiliência: Tolerância a dados sujos (NULLs, formatos ruins) sem quebrar o pipeline, deixando a limpeza para a Camada Silver. |

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS raw_data_volume;

In [0]:
%sql
-- Notebook 01: INGESTÃO DOS DADOS BRONZE (UC)

INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
VALUES ('bronze_ingestion', 'inicio', 0, 'iniciado', CURRENT_TIMESTAMP(), NULL);


USE CATALOG workspace;
USE pl_delivery_analysis;

-- -------------------------------------------------------------
-- A. FATO PRINCIPAL (ORDERS)
-- -------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_orders_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_orders_bronze (
  order_id STRING,
  created_at STRING,
  subtotal STRING,
  delivery_fee STRING,
  store_id STRING
);

COPY INTO pl_delivery_analysis.tbl_fact_orders_bronze
FROM (
  SELECT
    order_id,
    order_moment_created AS created_at,
    order_amount AS subtotal,
    order_delivery_fee AS delivery_fee,
    store_id
  FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/orders.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 


-- -----------------------------------------------------------
-- B. FATO LOGÍSTICO (DELIVERIES) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_deliveries_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_deliveries_bronze (
    order_id STRING,
    delivery_status STRING,
    driver_id STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_deliveries_bronze
FROM (
    SELECT
        delivery_order_id AS order_id,
        delivery_status,
        driver_id
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/deliveries.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 


-- -----------------------------------------------------------
-- C. FATO FINANCEIRO (PAYMENTS)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_payments_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_payments_bronze (
    order_id STRING,
    payment_method STRING,
    payment_amount STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_payments_bronze
FROM (
    SELECT
        payment_order_id AS order_id,
        payment_method,
        payment_amount
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/payments.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');


-- -----------------------------------------------------------
-- D. DIMENSÃO LOJA (STORES)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_stores_bronze;
CREATE TABLE pl_delivery_analysis.tbl_dim_stores_bronze (
    store_id STRING,
    store_segment STRING,
    hub_id STRING 
);
COPY INTO pl_delivery_analysis.tbl_dim_stores_bronze
FROM (
    SELECT
        store_id,
        store_segment,
        hub_id -- Corrigido: Usando hub_id que existe no CSV de stores
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/stores.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');


-- 3. Verificação Final (Confirma se o número de linhas carregou corretamente)
SELECT 'Orders' AS Tabela, COUNT(*) FROM pl_delivery_analysis.tbl_fact_orders_bronze
UNION ALL
SELECT 'Deliveries' AS Tabela, COUNT(*) FROM pl_delivery_analysis.tbl_fact_deliveries_bronze
UNION ALL
SELECT 'Payments' AS Tabela, COUNT(*) FROM pl_delivery_analysis.tbl_fact_payments_bronze
UNION ALL
SELECT 'Stores' AS Tabela, COUNT(*) FROM pl_delivery_analysis.tbl_dim_stores_bronze;

INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
SELECT 
    'bronze_ingestion', 
    'conclusao', 
    (SELECT COUNT(*) FROM tbl_fact_orders_bronze),
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    NULL;

In [0]:
%sql
-- INGESTÃO DA TABELA HUBS


INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
VALUES ('bronze_ingestion', 'inicio', 0, 'iniciado', CURRENT_TIMESTAMP(), NULL);

USE pl_delivery_analysis;

-- -----------------------------------------------------------
-- E. DIMENSÃO HUB (HUBS) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_hubs_bronze;

-- Cria a tabela
CREATE TABLE pl_delivery_analysis.tbl_dim_hubs_bronze (
    hub_id STRING,
    city STRING
);

-- Carrega os dados com o mapeamento e sintaxe CORRETOS
COPY INTO pl_delivery_analysis.tbl_dim_hubs_bronze
FROM (
    SELECT
        hub_id,
        hub_city AS city 
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/hubs.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- Verificação da Tabela Hubs
SELECT 'Dimensoes | Hubs' AS Tabela, COUNT(*)
FROM pl_delivery_analysis.tbl_dim_hubs_bronze;

INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
SELECT 
    'bronze_ingestion', 
    'conclusao', 
    (SELECT COUNT(*) FROM tbl_fact_orders_bronze),
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    NULL;

#### 3. Camada SILVER: Unificação e Cálculo da UE (Data Preparation - Transformation)
A Camada Silver é o Coração Analítico do pipeline, onde a lógica de negócio é aplicada e as métricas de Unit Economics são calculadas.

 Justificativas Técnicas
Unificação do Fato: Uso de INNER JOIN entre Pedidos, Entregas e Pagamentos na CTE pedidos_limpos.

Marketplace Contexto: Isso é fundamental, pois garante que a análise do P&L inclua apenas pedidos que completaram as três fases do two-sided marketplace (alocação de pedido, pagamento e logística).

Cálculo da UE: As premissas de negócio (Comissão, Repasse, Taxa Transacional) são definidas em uma CTE constantes e aplicadas via CROSS JOIN.

Reaproveitamento: Esta técnica permite alterar as premissas de negócio (ex: mudar a comissão de 18% para 20%) em um único lugar, sem reescrever o cálculo do P&L.

Limpeza de Dados: Conversão de tipos de dados (CAST(coluna AS DECIMAL(10, 2))) e filtragem de valores inválidos ou não entregues (delivery_status = 'DELIVERED').

#### Etapas & Benefícios

| Métrica Calculada | Fórmula | Benefício Estratégico |
| :--- | :--- | :--- |
| Receita de Comissão | subtotal_bruto * comissao_plataforma | Rentabilidade: Mede o lucro obtido do lado do fornecedor (loja). |
| COGS Logístico Simulado | delivery_fee_cliente * repasse_entregador | Controle de Custo: Mede o custo variável pago ao entregador (lado da logística). |
| Lucro Bruto Unitário | Receita Líquida - COGS Logístico - COGS Transação | Ouro do P&L: A métrica final que indica a Margem de Contribuição de cada pedido. |

In [0]:
%sql
-- Notebook 02: Modelagem e Unit Economics (Silver Layer) 

-- ===========================================================================
-- DECISÃO CRÍTICA: FILTRO DE PEDIDOS ENTREGUES
-- ===========================================================================
-- 
-- PROBLEMA: Deveríamos incluir todos os pedidos ou apenas os entregues?
--
-- OPÇÃO A: INNER JOIN (atual)
--   ✅ VANTAGENS: 
--     - Análise focada em transações completas
--     - Métricas de Unit Economics mais realistas
--     - Evita distorção por pedidos cancelados
--   ❌ DESVANTAGENS:
--     - Perde visibilidade do funil completo
--     - Pode mascarar problemas de cancelamento
--
-- OPÇÃO B: LEFT JOIN  
--   ✅ VANTAGENS:
--     - Visão completa de todos os pedidos
--     - Análise de taxa de conversão
--   ❌ DESVANTAGENS:
--     - Inclui pedidos que não geraram receita
--     - Distorce métricas de profitability
--
-- DECISÃO: INNER JOIN
-- JUSTIFICATIVA: O foco deste pipeline é Unit Economics (P&L por pedido).
-- Pedidos não entregues não geram receita, portanto não contribuem para
-- a análise de profitability que é o objetivo principal.
--
-- PARA ANÁLISE DE FUNIL: Criar pipeline separado focado em conversão.
-- ===========================================================================


INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
VALUES ('bronze_ingestion', 'inicio', 0, 'iniciado', CURRENT_TIMESTAMP(), NULL);

USE pl_delivery_analysis;

DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_pedidos_silver;

CREATE TABLE pl_delivery_analysis.tbl_fact_pedidos_silver AS
WITH constantes AS (
  -- Definindo premissas de negócio como constantes na CTE
  SELECT
    0.18 AS comissao_plataforma, -- 18%
    0.70 AS repasse_entregador, -- 70%
    0.02 AS taxa_transacao  -- 2%
),

pedidos_limpos AS (
  SELECT
    t1.order_id,
    t1.store_id,
    t2.driver_id,
    t3.payment_method,

    -- Conversão para tipos de dados corretos
    CAST(t1.subtotal AS DECIMAL(10, 2)) AS subtotal_bruto,
    CAST(t1.delivery_fee AS DECIMAL(10, 2)) AS delivery_fee_cliente,
    t1.created_at AS created_at_ts_str
    -- Você pode tentar converter para TIMESTAMP aqui, se necessário: TO_TIMESTAMP(t1.created_at) AS created_at_ts

  FROM
    pl_delivery_analysis.tbl_fact_orders_bronze t1
  INNER JOIN
    pl_delivery_analysis.tbl_fact_deliveries_bronze t2 ON t1.order_id = t2.order_id
  INNER JOIN
    pl_delivery_analysis.tbl_fact_payments_bronze t3 ON t1.order_id = t3.order_id
  WHERE
    t2.delivery_status = 'DELIVERED' -- <--- CORREÇÃO APLICADA: AGORA É EM CAIXA ALTA
    AND CAST(t1.subtotal AS DECIMAL(10, 2)) IS NOT NULL
)

SELECT
  p.order_id,
  p.store_id,
  p.driver_id,
  p.payment_method,
  p.subtotal_bruto,
  p.delivery_fee_cliente,
  p.created_at_ts_str,
  -- ------------------------------------------
  -- 2. Métricas de Receita (Plataforma)
  -- ------------------------------------------

  (p.subtotal_bruto + p.delivery_fee_cliente) AS gmv_total,
  (p.subtotal_bruto * c.comissao_plataforma) AS receita_comissao,
  (receita_comissao + (p.delivery_fee_cliente * (1 - c.repasse_entregador))) AS receita_liquida_plataforma,

  -- -------------------------------------------
  -- 3. Métricas de Custo (COGS Variável)
  -- -------------------------------------------
  (p.delivery_fee_cliente * c.repasse_entregador) AS cogs_logistico_simulado,
  ((p.subtotal_bruto + p.delivery_fee_cliente) * c.taxa_transacao) AS cogs_transacao_simulado,

  -- -----------------------------------------------------
  -- 4. UNIT ECONOMICS (Margem de Contribuição por Pedido)
  -- -----------------------------------------------------
  (receita_liquida_plataforma - cogs_logistico_simulado - cogs_transacao_simulado) AS lucro_bruto_unitario

FROM
  pedidos_limpos p
CROSS JOIN
  constantes c;
-- ---------------------------------------------------
-- 5. Verificação da Unit Economics (Primeiras linhas)
-- ---------------------------------------------------
SELECT
  order_id,
  lucro_bruto_unitario,
  gmv_total,
  receita_liquida_plataforma,
  cogs_logistico_simulado,
  cogs_transacao_simulado
FROM
  pl_delivery_analysis.tbl_fact_pedidos_silver
LIMIT 10;

INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
SELECT 
    'bronze_ingestion', 
    'conclusao', 
    (SELECT COUNT(*) FROM tbl_fact_orders_bronze),
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    NULL;

#### 4. Camada GOLD: Modelagem Dimensional (Modeling)

A Camada Gold é o resultado final, pronto para o consumo em ferramentas de BI (Power BI, Tableau). É estruturada em um Star Schema para otimizar o desempenho de queries e simplificar a análise.

 Justificativas Técnicas
Star Schema: Separa as métricas (Fato) dos atributos descritivos (Dimensão), acelerando drasticamente os relatórios de BI.

Enriquecimento de Dimensão: O LEFT JOIN na tbl_dim_store_gold (STORES + HUBS) foi crucial.

Contexto: Garante que todas as lojas sejam preservadas (mantendo todos os pedidos da Fato), enquanto anexa a informação geográfica (Cidade/Hub) que estava em outra tabela, criando uma dimensão de loja rica.

Dimensão Tempo (tbl_dim_time_gold): A conversão complexa de STRING para DATE usando TO_TIMESTAMP foi vital para extrair atributos temporais (year, quarter, month) que facilitam análises de tendência e sazonalidade no P&L.

#### Etapas & Benefícios

| Tabela Final | Conteúdo | Benefício para o BI |
| :--- | :--- | :--- |
| tbl_fato_delivery_gold | Métricas de UE e chaves de junção. | Performance: Permite que o Power BI some o lucro_bruto_unitario rapidamente, sem a necessidade de reprocessar cálculos. |
| tbl_dim_store_gold | store_id, store_segment, city. | Segmentação: Permite análises de P&L por segmento de loja (Premium vs. Low) ou por cidade, orientando estratégias de expansão regional. |
| tbl_dim_time_gold | date_key, year, quarter, month. | Tendência: Facilita a análise de séries temporais para identificar padrões de lucratividade (ex: P&L é maior no Q4?). |

In [0]:
%sql
-- Notebook 03: MODELAGEM DIMENSIONAL (GOLD LAYER) 


INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
VALUES ('bronze_ingestion', 'inicio', 0, 'iniciado', CURRENT_TIMESTAMP(), NULL);

USE pl_delivery_analysis;

-- Formato da string: M/d/yyyy h:mm:ss a (ex: 1/1/2021 2:30:00 PM)
SET spark.sql.legacy.timeParserPolicy = CORRECTED; -- Ajuste para lidar melhor com formatos complexos/ambíguos

-------------------------------------------------------------
-- FUNÇÃO DE CONVERSÃO: Define o formato de data/hora completo
-------------------------------------------------------------
-- Esta função converte a string completa em TIMESTAMP e depois extrai a data.
CREATE OR REPLACE FUNCTION convert_datetime_to_date(datetime_str STRING)
RETURNS DATE
RETURN TO_DATE(
    TO_TIMESTAMP(datetime_str, 'M/d/yyyy h:mm:ss a')
);

-------------------------------------------------------------
-- 1. CRIAÇÃO DA TABELA FATO (BASE DE DADOS) 
-------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fato_delivery_gold;

CREATE TABLE pl_delivery_analysis.tbl_fato_delivery_gold AS
SELECT
    p.order_id,
    p.store_id,
    p.driver_id,
    p.payment_method,

    -- Chave de Tempo: Usa a função de conversão completa
    convert_datetime_to_date(p.created_at_ts_str) AS order_date_key,

    p.subtotal_bruto,
    p.delivery_fee_cliente,
    p.gmv_total,
    p.receita_liquida_plataforma,
    p.cogs_logistico_simulado,
    p.cogs_transacao_simulado,
    p.lucro_bruto_unitario
FROM
    pl_delivery_analysis.tbl_fact_pedidos_silver p;

-------------------------------------------------------------
-- 2. CRIAÇÃO DA DIMENSÃO TEMPO (ESSENCIAL PARA ANÁLISE) 
-------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_time_gold;

CREATE TABLE pl_delivery_analysis.tbl_dim_time_gold AS
SELECT DISTINCT
    d.date_key,
    YEAR(d.date_key) AS year,
    MONTH(d.date_key) AS month,
    DAYOFMONTH(d.date_key) AS day,
    WEEKDAY(d.date_key) AS day_of_week_num,
    CASE 
        WHEN MONTH(d.date_key) IN (1, 2, 3) THEN 'Q1'
        WHEN MONTH(d.date_key) IN (4, 5, 6) THEN 'Q2'
        WHEN MONTH(d.date_key) IN (7, 8, 9) THEN 'Q3'
        ELSE 'Q4'
    END AS quarter
FROM (
    SELECT
        convert_datetime_to_date(created_at_ts_str) AS date_key
    FROM pl_delivery_analysis.tbl_fact_pedidos_silver
) d
WHERE d.date_key IS NOT NULL; -- Filtra datas que não puderam ser convertidas


-------------------------------------------------------------
-- 3. VERIFICAÇÃO FINAL DA CAMADA GOLD
-------------------------------------------------------------
-- A Dimensão Loja (tbl_dim_store_gold) já foi criada, não precisa recriar.
SELECT 'Fato Delivery Gold' AS Tabela, COUNT(*)
FROM pl_delivery_analysis.tbl_fato_delivery_gold
UNION ALL
SELECT 'Dim Store Gold' AS Tabela, COUNT(*)
FROM pl_delivery_analysis.tbl_dim_store_gold
UNION ALL
SELECT 'Dim Time Gold' AS Tabela, COUNT(*)
FROM pl_delivery_analysis.tbl_dim_time_gold;



-- ===========================================================================
-- POR QUE USAR ZORDER?
-- ===========================================================================
-- O ZORDER organiza os dados fisicamente no storage, agrupando registros
-- com valores similares nas colunas especificadas. Isso melhora MUITO a
-- performance de queries que filtram por essas colunas.
--
-- EXEMPLO: Se order_date_key está ZORDERed, queries como:
-- "WHERE order_date_key BETWEEN '2024-01-01' AND '2024-01-31'"
-- serão muito mais rápidas porque os dados estão fisicamente próximos.
-- ===========================================================================

OPTIMIZE pl_delivery_analysis.tbl_fato_delivery_gold
ZORDER BY (order_date_key, store_id);

-- ADICIONE COMENTÁRIOS ÀS TABELAS (documentação automática)
COMMENT ON TABLE pl_delivery_analysis.tbl_fato_delivery_gold IS '
Tabela fato principal com métricas de Unit Economics.
ZORDERed por order_date_key e store_id para performance.
Uso: Análise de P&L e profitability por pedido.
';

INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, registros_afetados, status, timestamp, erro)
SELECT 
    'bronze_ingestion', 
    'conclusao', 
    (SELECT COUNT(*) FROM tbl_fact_orders_bronze),
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    NULL;

### 5. Próximos Passos (Deployment)
O projeto está pronto para a fase de deployment e utilização. A conexão do Power BI às tabelas Gold permite que a equipe de estratégia e finanças monitore o Unit Cost/P&L em tempo real e tome decisões ágeis para otimizar a lucratividade do Marketplace.